# 🤖 네이버 블로그 포스팅 자동 생성기 (Jupyter Notebook)

**참고할 기사나 블로그 글의 URL을 입력**하면, AI 에이전트들이 협력하여 **네이버 SEO에 최적화된 블로그 포스트**를 자동으로 만들어 드립니다.

## 파이프라인 구조
1. **Researcher** → URL 콘텐츠 스크래핑
2. **SEO Specialist** → 네이버 SEO 전략 및 태그 생성
3. **Writer** → 블로그 포스트 작성
4. **Blog Indexer** → 블로그 품질 점수 계산
5. **Art Director** → DALL-E 이미지 생성

## 주요 특징
- 🎯 **대화형 인터페이스** - ipywidgets 기반 UI
- 🔧 **모듈화된 구조** - 각 에이전트 개별 실행 가능
- 📊 **실시간 진행 상황** - 단계별 피드백
- 🐛 **디버깅 도구** - 개별 에이전트 테스트

---

## 0. 필요한 라이브러리 설치 (optional)

- colab 환경에서 설치 필요

In [ ]:
!pip install -q langchain-community langchain-openai tavily-python beautifulsoup4 langgraph

print("✅ 필요한 라이브러리 설치 완료!")

## 1. 환경 설정 및 라이브러리 Import

In [ ]:
# 라이브러리 Import
import os
import requests
import json
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from typing import List, TypedDict
from IPython.display import display, Image, Markdown, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from langchain_community.document_compressors.llmlingua_filter import DEFAULT_LLM_LINGUA_INSTRUCTION

# LangChain 관련
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, END

# OpenAI (DALL-E용)
from openai import OpenAI

print("✅ 모든 라이브러리가 성공적으로 로드되었습니다!")

In [ ]:
# 환경 변수 로드
load_dotenv()

# 글로벌 변수 초기화
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

print(f"OPENAI_API_KEY 로드됨: {'✅' if OPENAI_API_KEY else '❌'}")
print(f"TAVILY_API_KEY 로드됨: {'✅' if TAVILY_API_KEY else '❌'}")

if not OPENAI_API_KEY or not TAVILY_API_KEY:
    print("\n⚠️ API 키가 설정되지 않았습니다. 다음 셀에서 직접 입력하세요.")

In [ ]:
# 모델 설정 (현재값 표시 및 수정 가능)
def setup_model_configs():
    """모델 설정을 위한 대화형 위젯"""
    global DEFAULT_LLM_MODEL, IMAGE_PROMPT_MODEL, IMAGE_GENERATE_MODEL

    # 현재 값을 기본값으로 하는 입력 위젯들
    llm_input = widgets.Text(
        value=DEFAULT_LLM_MODEL,
        placeholder='기본 LLM 모델명을 입력하세요',
        description='Default LLM:',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='400px')
    )

    image_prompt_input = widgets.Text(
        value=IMAGE_PROMPT_MODEL,
        placeholder='이미지 프롬프트 생성 모델명을 입력하세요',
        description='Image Prompt:',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='400px')
    )

    image_gen_input = widgets.Text(
        value=IMAGE_GENERATE_MODEL,
        placeholder='이미지 생성 모델명을 입력하세요',
        description='Image Generator:',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='400px')
    )

    update_button = widgets.Button(
        description="🔧 모델 설정 업데이트",
        button_style='info',
        icon='cog'
    )

    output = widgets.Output()

    def on_update_clicked(b):
        with output:
            clear_output(wait=True)

            # 글로벌 변수 업데이트
            global DEFAULT_LLM_MODEL, IMAGE_PROMPT_MODEL, IMAGE_GENERATE_MODEL

            DEFAULT_LLM_MODEL = llm_input.value or "gpt-4o"
            IMAGE_PROMPT_MODEL = image_prompt_input.value or "gpt-4o"
            IMAGE_GENERATE_MODEL = image_gen_input.value or "dall-e-3"

            print("✅ 모델 설정이 업데이트되었습니다!")
            print(f"   - Default LLM: {DEFAULT_LLM_MODEL}")
            print(f"   - Image Prompt Model: {IMAGE_PROMPT_MODEL}")
            print(f"   - Image Generator: {IMAGE_GENERATE_MODEL}")

    update_button.on_click(on_update_clicked)

    display(widgets.VBox([
        widgets.HTML("<h3>⚙️ 모델 설정</h3>"),
        widgets.HTML("<p>현재 설정값이 표시됩니다. 필요시 수정 후 업데이트 버튼을 클릭하세요.</p>"),
        llm_input,
        image_prompt_input,
        image_gen_input,
        update_button,
        output
    ]))

# 초기 모델 설정값
DEFAULT_LLM_MODEL = "gpt-4o"
IMAGE_PROMPT_MODEL = "gpt-4o"
IMAGE_GENERATE_MODEL = "dall-e-3"

# 모델 설정 위젯 표시
setup_model_configs()

## 2. API 키 설정 (필요시 직접 입력)

In [ ]:
# API 키 입력 위젯 (보안 개선)
def setup_api_keys():
    """API 키를 안전하게 설정하고 검증하는 함수"""

    openai_input = widgets.Password(
        value='',  # 빈 값으로 시작
        placeholder='OpenAI API Key를 입력하세요',
        description='OpenAI:',
        disabled=False
    )

    tavily_input = widgets.Password(
        value='',  # 빈 값으로 시작
        placeholder='Tavily API Key를 입력하세요',
        description='Tavily:',
        disabled=False
    )

    save_button = widgets.Button(
        description="💾 API Keys 저장",
        button_style='success',
        icon='check'
    )

    output = widgets.Output()

    def on_save_clicked(b):
        with output:
            clear_output(wait=True)

            global OPENAI_API_KEY, TAVILY_API_KEY

            # 키 검증 및 저장 (환경변수 직접 조작 제거)
            if openai_input.value:
                OPENAI_API_KEY = openai_input.value
                print("✅ OpenAI API Key 설정됨")
                openai_input.value = '' # 저장 후 값 비우기

            if tavily_input.value:
                TAVILY_API_KEY = tavily_input.value
                print("✅ Tavily API Key 설정됨")
                tavily_input.value = '' # 저장 후 값 비우기


            if OPENAI_API_KEY and TAVILY_API_KEY:
                print("\n🎉 모든 API 키가 설정되었습니다! 다음 단계로 진행하세요.")
                print("\n⚠️  보안 알림: API 키는 현재 세션 메모리에만 저장됩니다.")
                print("     노트북을 재시작하면 다시 입력해야 합니다.")
            elif OPENAI_API_KEY or TAVILY_API_KEY:
                 print("\n⚠️ 일부 API 키만 설정되었습니다. 필요한 키를 모두 설정하세요.")
            else:
                 print("\n❌ API 키가 설정되지 않았습니다.")


    save_button.on_click(on_save_clicked)

    display(widgets.VBox([
        widgets.HTML("<h3>🔑 API 키 설정 (보안 강화)</h3>"),
        widgets.HTML("<p>⚠️ API 키는 현재 세션 메모리에만 임시로 저장되며, 노트북 파일에 저장되지 않습니다.</p>"),
        openai_input,
        tavily_input,
        save_button,
        output
    ]))

setup_api_keys()

## 3. 유틸리티 함수 정의

In [ ]:
# 에이전트 상태 정의
class AgentState(TypedDict):
    url: str
    scraped_content: str
    seo_analysis: str
    seo_tags: List[str]
    draft_post: str
    final_title: str
    final_subheadings: List[str]
    final_post: str
    image_prompt: str
    image_url: str
    blog_index: int
    messages: List[BaseMessage]

print("✅ AgentState 정의 완료")

In [ ]:
# 웹 스크래핑 함수
def scrape_web_content(url: str) -> str:
    """지정된 URL의 웹 콘텐츠를 스크래핑하여 텍스트를 반환합니다."""
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # 본문 콘텐츠 위주로 추출
        main_content = soup.find('main') or soup.find('article') or soup.body
        if main_content:
            # 불필요한 태그 제거
            for tag in main_content(['nav', 'footer', 'script', 'style', 'aside', 'form']):
                tag.decompose()
            text = main_content.get_text(separator='\n', strip=True)
            return text
        return "콘텐츠를 추출할 수 없습니다."
    except requests.RequestException as e:
        return f"URL 요청 중 오류 발생: {e}"
    except Exception as e:
        return f"콘텐츠 스크래핑 중 오류 발생: {e}"

print("✅ 웹 스크래핑 함수 정의 완료")

In [ ]:
# LLM 인스턴스 생성 헬퍼 (보안 개선)
def get_llm(model: str = DEFAULT_LLM_MODEL, temperature: float = 0.7):
    """OpenAI API 키를 확인하고 ChatOpenAI 인스턴스를 반환합니다."""
    global OPENAI_API_KEY

    # 글로벌 변수에서 직접 API 키 확인
    if not OPENAI_API_KEY:
        print("❌ OpenAI API Key가 설정되지 않았습니다.")
        return None

    try:
        # API 키를 직접 전달 (환경변수 조작 없이)
        return ChatOpenAI(model=model, temperature=temperature, api_key=OPENAI_API_KEY)
    except Exception as e:
        print(f"❌ OpenAI LLM 초기화 실패: {e}")
        return None

print("✅ LLM 헬퍼 함수 정의 완료 (보안 강화)")

## 4. 에이전트 노드 함수들

In [ ]:
# 4.1. 리서처 에이전트 (URL 스크래핑)
def researcher_node(state: AgentState):
    """입력된 URL의 콘텐츠를 스크래핑하여 다음 단계로 전달합니다."""
    print("▶️ 리서처 에이전트: URL 콘텐츠 분석 시작...")
    url = state['url']
    scraped_content = scrape_web_content(url)

    if "오류 발생" in scraped_content or "추출할 수 없습니다" in scraped_content:
        print(f"❌ 콘텐츠를 가져오는 데 실패했습니다: {scraped_content}")
        return {
            "scraped_content": f"분석 실패: {scraped_content}",
            "messages": [HumanMessage(content=f"URL 스크래핑 실패: {url}")]
        }

    print("✅ 리서처 에이전트: 콘텐츠 분석 완료!")
    print(f"📄 추출된 콘텐츠 길이: {len(scraped_content)} 문자")

    return {
        "scraped_content": scraped_content,
        "messages": [HumanMessage(content=f"URL '{url}'의 콘텐츠 분석 완료.")]
    }

print("✅ 리서처 에이전트 정의 완료")

In [ ]:
# 4.2. SEO 전문가 에이전트
def seo_specialist_node(state: AgentState):
    """스크랩된 콘텐츠를 기반으로 네이버 SEO 전략을 분석하고 태그를 생성합니다."""
    print("▶️ SEO 전문가 에이전트: 네이버 SEO 전략 분석 중...")
    scraped_content = state['scraped_content']

    # 네이버 SEO 트렌드 검색
    search_query = "2025년 네이버 블로그 SEO 최적화 전략"

    global TAVILY_API_KEY
    if not TAVILY_API_KEY:
        print("❌ Tavily API Key가 설정되어 있지 않습니다.")
        return {"seo_analysis": "Tavily API Key 없음", "seo_tags": []}

    # Tavily 도구 인스턴스화
    try:
        tavily_tool = TavilySearchResults(max_results=5, tavily_api_key=TAVILY_API_KEY)
        seo_trends = tavily_tool.invoke({"query": search_query})
        print("✅ SEO 트렌드 검색 완료")
    except Exception as e:
        print(f"❌ Tavily 검색 도구 오류: {e}")
        seo_trends = ""

    prompt = ChatPromptTemplate.from_messages([
        ("system",
         """당신은 15년 경력의 네이버 블로그 SEO 전문가입니다.
         당신의 목표는 주어진 원본 콘텐츠와 최신 SEO 트렌드 정보를 바탕으로, 네이버 검색에 최적화된 블로그 포스트 전략을 수립하는 것입니다.

         지침:
         1. 원본 콘텐츠의 핵심 주제와 주요 키워드를 파악합니다.
         2. 최신 네이버 SEO 트렌드를 참고하여, 어떤 키워드와 주제를 강조해야 할지 결정합니다.
         3. 사용자들이 검색할 만한 매력적이고 구체적인 롱테일 키워드를 포함한 제목과 소제목 아이디어를 제안합니다.
         4. 네이버 블로그에 사용될 SEO에 가장 효과적인 태그 30개를 정확히 추출하여 리스트 형태로 제공합니다.
         5. 모든 결과물은 한국어로 작성해야 합니다.

         결과는 다음 형식으로 정리해주세요:

         [분석 및 전략]
         - (여기에 콘텐츠를 기반으로 한 SEO 전략과 키워드 분석 내용을 서술)

         [추천 태그]
         태그1, 태그2, 태그3, 태그4, 태그5, 태그6, 태그7, 태그8, 태그9, 태그10,
         태그11, 태그12, 태그13, 태그14, 태그15, 태그16, 태그17, 태그18, 태그19, 태그20,
         태그21, 태그22, 태그23, 태그24, 태그25, 태그26, 태그27, 태그28, 태그29, 태그30,
         """),
        ("human",
         "**최신 네이버 SEO 트렌드:**\n{seo_trends}\n\n"
         "**분석할 원본 콘텐츠:**\n{scraped_content}"),
    ])

    llm = get_llm()
    if llm is None:
        print("❌ LLM을 초기화할 수 없습니다.")
        return {"seo_analysis": "LLM 없음", "seo_tags": []}

    chain = prompt | llm
    response = chain.invoke({
        "seo_trends": seo_trends,
        "scraped_content": scraped_content[:4000]
    })

    # 결과 파싱
    analysis_text = response.content
    try:
        tags_part = analysis_text.split("[추천 태그]")[1].strip()
        tags = [tag.strip() for tag in tags_part.split(", ")]
    except IndexError:
        print("⚠️ 태그 파싱 실패, 기본 태그 사용")
        tags = ["블로그", "포스팅", "정보", "팁", "가이드"]

    print(f"✅ SEO 전문가 에이전트: 전략 분석 및 {len(tags)}개 태그 생성 완료!")
    return {
        "seo_analysis": analysis_text,
        "seo_tags": tags
    }

print("✅ SEO 전문가 에이전트 정의 완료")

In [ ]:
# 4.3. 작성가 에이전트
def writer_node(state: AgentState):
    """SEO 분석 결과를 바탕으로 실제 블로그 포스트 초안을 작성합니다."""
    print("▶️ 작성가 에이전트: 블로그 포스트 초안 작성 중...")
    scraped_content = state['scraped_content']
    seo_analysis = state['seo_analysis']

    # 제목 생성
    title_prompt = ChatPromptTemplate.from_messages([
        ("system",
         """당신은 네이버 블로그 SEO 전문가입니다. 주어진 콘텐츠와 SEO 분석을 바탕으로 클릭을 유도하는 매력적인 제목을 만드는 것이 임무입니다.

         요구사항:
         - SEO 키워드를 자연스럽게 포함
         - 호기심을 자극하는 표현 사용
         - 네이버 검색에 최적화된 길이 (30-40자)
         - 매력적이고 클릭률이 높은 하나의 제목 제안

         결과는 제목만 출력하세요 (추가 설명 없이).
         """),
        ("human",
         "**SEO 전문가 분석 및 전략:**\n{seo_analysis}\n\n"
         "**참고할 원본 콘텐츠:**\n{scraped_content}"),
    ])

    llm = get_llm()
    if llm is None:
        print("❌ LLM을 초기화할 수 없습니다.")
        return {"draft_post": "LLM 없음", "final_title": "", "final_subheadings": []}

    title_chain = title_prompt | llm
    main_title = title_chain.invoke({
        "seo_analysis": seo_analysis,
        "scraped_content": scraped_content[:4000]
    }).content.strip()

    print(f"📝 제목 생성 완료: {main_title}")

    # 본문 작성
    prompt = ChatPromptTemplate.from_messages([
        ("system",
         """당신은 사람들의 시선을 사로잡는 글을 쓰는 전문 블로그 작가입니다. 네이버 블로그 플랫폼의 특성을 잘 이해하고 있습니다.
         당신의 임무는 주어진 제목과 SEO 전문가의 분석 자료를 바탕으로, 독자들이 쉽게 읽고 공감할 수 있는 매력적인 블로그 포스트를 마크다운 형식으로 작성하는 것입니다.

         작성 가이드라인:
         1. **제목:** 주어진 제목을 `#`으로 시작하여 사용하세요.
         2. **소개:** 독자의 흥미를 유발하고 글을 계속 읽고 싶게 만드는 도입부를 작성하세요.
         3. **본문:** SEO 전문가가 제안한 소제목 아이디어를 활용하여 여러 개의 소제목(`##`)으로 문단을 나누세요. 각 문단은 원본 콘텐츠의 내용을 바탕으로 하되, 더 친근하고 이해하기 쉬운 문체로 재구성합니다. 이모지를 적절히 사용하여 가독성을 높여주세요.
         4. **결론:** 글의 내용을 요약하고, 독자에게 행동을 유도하거나 긍정적인 메시지를 전달하며 마무리하세요.
         5. **스타일:** 전체적으로 친근하고 대화하는 듯한 톤앤매너를 유지하고, 각 토픽은 500자 이상 1000자 이하로 작성해주세요
         """),
        ("human",
         "**사용할 제목:**\n{title}\n\n"
         "**SEO 전문가 분석 및 전략:**\n{seo_analysis}\n\n"
         "**참고할 원본 콘텐츠:**\n{scraped_content}"),
    ])

    chain = prompt | llm
    draft_post = chain.invoke({
        "title": main_title,
        "seo_analysis": seo_analysis,
        "scraped_content": scraped_content[:4000]
    }).content

    # 소제목 추출
    lines = draft_post.split('\n')
    subheadings = []
    for line in lines:
        if line.startswith('## '):
            subheadings.append(line.replace('## ', '').strip())

    print(f"✅ 작성가 에이전트: 포스트 초안 작성 완료! (소제목 {len(subheadings)}개)")
    return {
        "draft_post": draft_post,
        "final_title": main_title,
        "final_subheadings": subheadings
    }

print("✅ 작성가 에이전트 정의 완료")

In [ ]:
# 4.4. 블로그 지수 계산 에이전트
def blog_indexer_node(state: AgentState):
    """블로그 지수를 계산하는 에이전트"""
    print("📊 블로그 지수 계산 중...")

    draft_post = state["draft_post"]

    prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 블로그 콘텐츠 전문가입니다. 주어진 블로그 게시물을 분석하여 블로그 지수(Blog Index)를 계산해주세요.

다음 10개 항목을 각각 0-10점으로 평가하여 총 100점 만점으로 채점하고, 각 항목별 평가 근거와 개선점을 제시해주세요.

## 평가 기준

### 1. 검색 최적화 제목 작성 (10점)
- 핵심 키워드가 앞부분에 위치하는가? (3점)
- 숫자, 시간, 지역명을 활용했는가? (3점)
- 클릭을 유도하는 감정 단어가 포함되어 있는가? (4점)

### 2. 첫 문단에서 핵심 요약 (10점)
- 3줄 이내에 글 전체를 이해할 수 있도록 정리되었는가? (6점)
- 질문형으로 시작하여 호기심을 자극하는가? (4점)

### 3. 독자 공감 포인트 확보 (10점)
- 실제 사례, 경험담, 에피소드가 포함되어 있는가? (6점)
- "저도 처음엔 몰랐는데…" 같은 톤으로 신뢰감을 형성하는가? (4점)

### 4. 본문 구조화 (10점)
- 소제목에 키워드가 포함되어 있는가? (4점)
- 목록/번호를 활용하여 가독성을 강화했는가? (3점)
- 긴 문장을 2~3줄로 끊어 썼는가? (3점)

### 5. 꾸준한 구독자 유입을 위한 시리즈화 (10점)
- 단발성이 아닌 연재 시리즈로 구성되었는가? (5점)
- 후속 편이나 관련 콘텐츠를 예고하고 있는가? (5점)

### 6. 내부 링크 & 외부 링크 전략 (10점)
- 블로그 내 다른 글로 자연스럽게 연결되어 있는가? (5점)
- 신뢰할 수 있는 외부 출처 1~2개를 인용하고 있는가? (5점)

### 7. 이미지 활용법 (10점)
- 글당 최소 3장 이상의 이미지를 사용했는가? (4점)
- 핵심 키워드를 포함한 그림 파일명을 작성했는가? (3점)
- ALT 텍스트에 설명을 추가했는가? (3점)

### 8. CTA(Call To Action) 삽입 (10점)
- 공감/구독/이웃추가를 유도하는 문구가 있는가? (5점)
- 댓글을 유도하는 질문이 포함되어 있는가? (5점)

### 9. 메타데이터와 태그 최적화 (10점)
- 글의 카테고리, 해시태그가 키워드와 일치하는가? (5점)
- 핵심 키워드 3~5개에 집중하여 태그를 설정했는가? (5점)

### 10. 콘텐츠 차별화 요소 추가 (10점)
- 직접 촬영한 사진, 인포그래픽, 표, 차트를 활용했는가? (5점)
- 단순 요약형이 아닌 경험+인사이트를 담아 독창성을 강화했는가? (5점)

## 출력 형식
반드시 다음과 같은 JSON 형식으로 출력해주세요:

{{"blog_index": {{"total_score": 85}}}}

위의 평가 기준에 따라 주어진 블로그 게시물을 분석하고, 반드시 위의 JSON 형식으로 블로그 지수를 계산해주세요."""),
        ("human", "다음 블로그 게시물의 블로그 지수를 분석해주세요:\n\n{draft_post}")
    ])

    llm = get_llm()
    if llm is None:
        print("❌ LLM을 초기화할 수 없습니다.")
        return {"blog_index": {"error": "LLM 없음"}}

    try:
        chain = prompt | llm
        response = chain.invoke({"draft_post": draft_post})

        # JSON 응답 파싱
        content = response.content

        # JSON 부분만 추출
        if "```json" in content:
            json_start = content.find("```json") + 7
            json_end = content.find("```", json_start)
            json_content = content[json_start:json_end].strip()
        elif "{" in content and "}" in content:
            json_start = content.find("{")
            json_end = content.rfind("}") + 1
            json_content = content[json_start:json_end]
        else:
            json_content = content

        blog_index_full = json.loads(json_content)
        total_score = blog_index_full.get("blog_index", {}).get("total_score", 0)
        print(f"✅ 블로그 지수 계산 완료: {total_score}점")
        return {"blog_index": int(total_score)}

    except Exception as e:
        print(f"❌ 블로그 지수 계산에 실패했습니다: {e}")
        return {"blog_index": 0}  # 오류 시 0점 반환

print("✅ 블로그 지수 계산 에이전트 정의 완료")

In [ ]:
# 4.5. 아트 디렉터 에이전트 (보안 개선)
def art_director_node(state: AgentState):
    """블로그 제목과 내용을 기반으로 DALL-E를 사용하여 이미지를 생성합니다."""
    print("▶️ 아트 디렉터 에이전트: 대표 이미지 생성 중...")
    title = state['final_title']
    draft_post = state['draft_post']

    # DALL-E 프롬프트 생성
    prompt_generator_llm = get_llm(model=IMAGE_PROMPT_MODEL, temperature=0.7)
    if prompt_generator_llm is None:
        print("❌ LLM을 초기화할 수 없습니다.")
        return {"image_prompt": "", "image_url": "이미지 생성 실패 - LLM 없음"}

    prompt_template = ChatPromptTemplate.from_messages([
        ("system", "당신은 창의적인 아트 디렉터입니다. 블로그 포스트의 제목과 내용을 바탕으로, DALL-E 3가 이미지를 생성할 수 있는 가장 효과적이고 상세한 영어 프롬프트를 한 문장으로 생성해야 합니다."),
        ("human", f"블로그 제목: {title}\n\n블로그 내용 요약:\n{draft_post[:500]}\n\n위 내용을 대표할 수 있는 이미지 프롬프트를 영어로 만들어주세요.")
    ])

    chain = prompt_template | prompt_generator_llm
    image_prompt = chain.invoke({}).content
    print(f"🎨 이미지 프롬프트 생성 완료: {image_prompt[:50]}...")

    # 이미지 생성 (보안 개선)
    global OPENAI_API_KEY
    if not OPENAI_API_KEY:
        print("❌ OpenAI API Key가 설정되지 않았습니다.")
        return {"image_prompt": image_prompt, "image_url": "이미지 생성 실패 - API Key 없음"}

    try:
        # API 키를 직접 전달 (환경변수 조작 없이)
        client = OpenAI(api_key=OPENAI_API_KEY)
        response = client.images.generate(
            model="dall-e-3",
            prompt=image_prompt,
            size="1024x1024",
            quality="standard",
            n=1,
        )
        image_url = response.data[0].url
        print("✅ 아트 디렉터 에이전트: 이미지 생성 완료!")
        return {"image_prompt": image_prompt, "image_url": image_url}
    except Exception as e:
        print(f"❌ 이미지 생성에 실패했습니다: {e}")
        return {"image_prompt": image_prompt, "image_url": "이미지 생성 실패"}

print("✅ 아트 디렉터 에이전트 정의 완료 (보안 강화)")

## 5. 워크플로우 그래프 빌드

In [ ]:
# 워크플로우 그래프 빌드 함수
def build_graph():
    """LangGraph StateGraph를 구성하고 반환합니다."""
    workflow = StateGraph(AgentState)

    # 노드 추가
    workflow.add_node("researcher", researcher_node)
    workflow.add_node("seo_specialist", seo_specialist_node)
    workflow.add_node("writer", writer_node)
    workflow.add_node("blog_indexer", blog_indexer_node)
    workflow.add_node("art_director", art_director_node)

    # 엣지 연결 (순차적 실행)
    workflow.set_entry_point("researcher")
    workflow.add_edge("researcher", "seo_specialist")
    workflow.add_edge("seo_specialist", "writer")
    workflow.add_edge("writer", "blog_indexer")
    workflow.add_edge("blog_indexer", "art_director")
    workflow.add_edge("art_director", END)

    return workflow.compile()

print("✅ 워크플로우 그래프 빌드 함수 정의 완료")

# 그래프 인스턴스 생성
app = build_graph()
print("✅ 워크플로우 그래프 인스턴스 생성 완료")

## 6. 인터랙티브 실행 인터페이스

URL을 입력하고 버튼을 클릭하면 자동으로 파이프라인이 실행됩니다.

Example :
- input: https://letspl.me/quest/2135?utm_source=letspl&utm_medium=email&utm_campaign=insight_recommend
- output: https://blog.naver.com/marantz2000/223977912972

In [ ]:
# 간단한 실행 인터페이스
# URL을 입력 후 아래 셀을 실행하세요

# URL 입력 (여기에 직접 URL을 입력하세요)
current_url = ""  # 예: "https://example.com/article"

# URL 입력 위젯
from IPython.display import display
import ipywidgets as widgets

url_input = widgets.Text(
    value='https://letspl.me/quest/2135?utm_source=letspl&utm_medium=email&utm_campaign=insight_recommend',
    placeholder='https://... (분석할 기사 또는 블로그 URL을 입력하세요)',
    description='URL:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

display(url_input)

print("👆 위에 URL을 입력한 후 아래 '파이프라인 실행' 셀을 실행하세요.")

In [ ]:
# 파이프라인 실행
# 위 셀에서 URL을 입력한 후 이 셀을 실행하세요

def run_pipeline():
    """파이프라인을 실행하는 함수"""
    # URL 가져오기
    url = url_input.value.strip()

    if not url:
        print("❌ URL을 입력해주세요!")
        return

    global OPENAI_API_KEY, TAVILY_API_KEY
    if not OPENAI_API_KEY or not TAVILY_API_KEY:
        print("❌ API 키가 설정되지 않았습니다. 섹션 2에서 API 키를 설정하세요.")
        return

    print(f"🎯 분석할 URL: {url}")
    print("\n🤖 AI 멀티에이전트가 작업을 시작합니다... 잠시만 기다려주세요.")
    print("="*60)

    try:
        # 초기 상태 설정
        initial_state = {"url": url, "messages": []}

        # 그래프 실행
        final_state = app.invoke(initial_state)

        print("\n🎉 모든 에이전트 작업 완료!")
        print("="*60)

        # 결과 표시
        display_results(final_state)

        # 결과를 전역 변수에 저장
        global current_results
        current_results = final_state

        return final_state

    except Exception as e:
        print(f"❌ 파이프라인 실행 중 오류 발생: {e}")
        return None

def display_results(results):
    """결과를 표시하는 함수"""
    print("\n✨" + "="*20 + " 최종 결과물 " + "="*20 + "✨")
    print()

    # 1. 생성된 이미지 표시
    if results.get("image_url") and "이미지 생성 실패" not in results["image_url"]:
        print("🖼️ 생성된 대표 이미지:")
        try:
            display(Image(url=results["image_url"], width=400))
            print(f"프롬프트: {results.get('image_prompt', 'N/A')[:100]}...")
        except Exception as e:
            print(f"이미지 표시 오류: {e}")
    else:
        print("⚠️ 대표 이미지를 생성하지 못했습니다.")

    print("\n" + "-"*50)

    # 2. 추천 제목 표시
    print("📝 추천 제목:")
    print(f"   {results.get('final_title', '제목 생성 실패')}")

    print("\n" + "-"*50)

    # 3. 블로그 지수 표시
    blog_index_score = results.get('blog_index')
    if blog_index_score and isinstance(blog_index_score, (int, float)):
        print("📊 블로그 지수:")

        # 등급 계산
        if blog_index_score >= 90:
            grade, desc = "S", "최우수"
        elif blog_index_score >= 80:
            grade, desc = "A", "우수"
        elif blog_index_score >= 70:
            grade, desc = "B", "양호"
        elif blog_index_score >= 60:
            grade, desc = "C", "보통"
        else:
            grade, desc = "D", "개선 필요"

        print(f"   총점: {blog_index_score}/100점")
        print(f"   등급: {grade} ({desc})")

    print("\n" + "-"*50)

    # 4. 추천 태그 표시
    print("🔖 추천 태그 (복사해서 사용하세요):")
    tags = results.get('seo_tags', [])
    if tags:
        tags_str = ", ".join([f"#{tag}" for tag in tags[:20]])
        print(f"   {tags_str}")
        if len(tags) > 20:
            print(f"   ... 및 {len(tags)-20}개 추가")

    print("\n" + "="*60)

    # 5. 블로그 포스트 표시
    print("\n✍️ 완성된 블로그 포스트:")
    print("-"*60)
    draft_post = results.get('draft_post', '포스트 생성 실패')
    if draft_post and draft_post != '포스트 생성 실패':
        display(Markdown(draft_post))

# 전역 변수 초기화
current_results = {}

# 파이프라인 실행
run_pipeline()

## 7. 디버깅 및 상세 분석

In [ ]:
# 디버깅 및 상세 분석 표시
def show_debug_details():
    """에이전트별 상세 작업 내용을 표시합니다."""
    global current_results

    if not current_results:
        print("📋 실행 결과가 없습니다. 먼저 파이프라인을 실행하세요.")
        return

    print("🔍" + "="*15 + " 에이전트 작업 상세 내용 " + "="*15 + "🔍")

    # 1. 스크래핑된 콘텐츠 (처음 500자)
    print("\n📄 스크래핑된 원본 콘텐츠 (처음 500자):")
    print("-"*40)
    scraped = current_results.get('scraped_content', '없음')
    print(scraped[:500] + "..." if len(scraped) > 500 else scraped)

    # 2. SEO 전문가 분석
    print("\n🔍 SEO 전문가 분석:")
    print("-"*40)
    seo_analysis = current_results.get('seo_analysis', '분석 내용 없음')
    if len(seo_analysis) > 1000:
        print(seo_analysis[:1000] + "\n... (생략)")
    else:
        print(seo_analysis)

    # 3. 추출된 소제목들
    print("\n📑 추출된 소제목들:")
    print("-"*40)
    subheadings = current_results.get('final_subheadings', [])
    if subheadings:
        for i, heading in enumerate(subheadings, 1):
            print(f"{i}. {heading}")
    else:
        print("소제목 없음")

    # 4. 메시지 로그
    print("\n💬 에이전트 메시지 로그:")
    print("-"*40)
    messages = current_results.get('messages', [])
    if messages:
        for msg in messages:
            if hasattr(msg, 'content'):
                print(f"- {msg.content}")
    else:
        print("메시지 없음")

# 상세 내용 표시 (접기 가능)
debug_button = widgets.Button(
    description="🔍 상세 분석 보기",
    button_style='info',
    icon='search'
)

debug_output = widgets.Output()

def on_debug_clicked(b):
    with debug_output:
        clear_output(wait=True)
        show_debug_details()

debug_button.on_click(on_debug_clicked)

display(widgets.VBox([
    debug_button,
    debug_output
]))

---

## 🎉 완료!

**네이버 블로그 포스팅 자동 생성기** Jupyter Notebook 버전

### 사용법:
1. **API 키 설정** (섹션 2)
2. **URL 입력 및 실행** (섹션 6)
3. **파이프라인 실행** (섹션 7)
4. **결과 확인** (섹션 8)

### 주요 기능:
- ✅ **대화형 위젯** - ipywidgets 기반 사용자 인터페이스
- ✅ **모듈화된 구조** - 각 에이전트를 개별적으로 실행 가능
- ✅ **실시간 피드백** - 각 단계별 진행 상황 표시
- ✅ **디버깅 도구** - 상세 분석 및 개별 테스트 기능
- ✅ **결과 시각화** - 이미지, 텍스트, 메트릭 통합 표시

### 실행 명령:
```bash
# Jupyter Lab 실행
uv run python -m jupyterlab blog-agent.ipynb

# 또는 Jupyter Notebook 실행
uv run python -m notebook blog-agent.ipynb
```

**Happy Blogging! 🚀**